### Answering questions using store database

---

In [1]:
import store as s
import pandas as pd
from traceback import print_exc as pe

# For testing, python will not reload a module that is already loaded
# unless you force it to
from importlib import reload
reload(s);

In [5]:
# Create view to separate the year-month from the day for simpler grouping.
sql = "DROP VIEW IF EXISTS vOrderDatesParsed"
s.RunAction(sql)
sql = """
CREATE VIEW vOrderDatesParsed as

SELECT *,
    substr(date,1,4) as year,
    substr(date,6,2) as month,
    substr(date,9,2) as day,
    substr(date,1,7) as year_month
FROM tOrder
;"""
s.RunAction(sql)

In [6]:
# Create a view that calculates the total amount of sales per year-month for simpler queries for some questions below.
sql = "DROP VIEW IF EXISTS vSalesByYearMonth"
s.RunAction(sql)
sql = """
CREATE VIEW vSalesByYearMonth as
SELECT year_month, SUM(unit_price*qty) as Sales
FROM vOrderDatesParsed
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
GROUP BY year_month 
ORDER BY year_month ASC
;"""
s.RunAction(sql)

---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year_month (as a single column, like YYYY-MM)
 - Sales: total sales for the month (i.e., sum of qty * unit price)
 - NumOrders: number of orders placed for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - OrdersPerCust: average number of orders per customer (i.e. NumOrders/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerOrder: average sales per order (i.e. Sales/NumOrders)

Sort the results should by year_month, in ascending order.

_Hint: Watch out for integer division!_

In [8]:
sql = """
WITH SalesOrdersCust as
(
SELECT year_month, SUM(unit_price*qty) as Sales, COUNT(DISTINCT(order_id)) as NumOrders, COUNT(DISTINCT(cust_id)) as NumCust
FROM vOrderDatesParsed
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
GROUP BY year_month
ORDER BY year_month ASC
)
SELECT *, (1.0*NumOrders/NumCust) as OrdersPerCust, (1.0*Sales/NumCust) as SalesPerCust, (1.0*Sales/NumOrders) as SalesPerOrder
FROM SalesOrdersCust
;"""
s.RunQuery(sql)

,year_month,Sales,NumOrders,NumCust,OrdersPerCust,SalesPerCust,SalesPerOrder
0,2019-01,68464.61,91,85,1.070588,805.466000,752.358352
1,2019-02,55560.32,80,73,1.095890,761.100274,694.504000
2,2019-03,19191.68,51,49,1.040816,391.666939,376.307451
3,2019-04,20912.07,48,46,1.043478,454.610217,435.668125
4,2019-05,11973.34,50,46,1.086957,260.290000,239.466800
5,2019-06,13737.30,43,41,1.048780,335.056098,319.472093
6,2019-07,22095.05,45,41,1.097561,538.903659,491.001111
7,2019-08,15675.05,51,49,1.040816,319.898980,307.353922
8,2019-09,9360.38,40,39,1.025641,240.009744,234.009500
9,2019-10,48411.35,58,51,1.137255,949.242157,834.678448


---
2) In which month did we have the lowest total sales?

Return one record with:
- year_month (as a single column, with values like YYYY-MM)
- sales (sum of qty * unit_price)

In [9]:
sql = """
SELECT year_month, MIN(Sales) as Sales
FROM vSalesByYearMonth
;"""
s.RunQuery(sql)

,year_month,Sales
0,2019-09,9360.38


---

3. In the month determined from the previous question, generate a list of our total sales by state.  Make sure that all states are included, even if they have no sales (50 states + PR and DC = 52 total records).

Return:

- The two-letter state abbreviation (st)
- Total sales for the month in question

Order the results by the state abbreviation, ascending

In [10]:
sql = """
WITH 
LowestSalesMonth as
    (
    SELECT year_month, MIN(Sales) as Sales
    FROM vSalesByYearMonth
    ),
StatesWithSales as
    (
    SELECT st, SUM(qty*unit_price) as Sales
    FROM vOrderDatesParsed
    JOIN tOrderDetail USING(order_id)
    JOIN tProd USING(prod_id)
    JOIN tCust USING(cust_id)
    JOIN tZip USING(zip)
    WHERE year_month = (SELECT year_month FROM LowestSalesMonth)
    GROUP BY st
    )
SELECT st, IFNULL(Sales,0) as TotalSales
FROM tState
LEFT JOIN StatesWithSales USING(st)
ORDER BY st ASC
;"""
s.RunQuery(sql)

,st,TotalSales
0,AK,0.00
1,AL,28.00
2,AR,0.00
3,AZ,0.00
4,CA,557.00
5,CO,0.00
6,CT,0.00
7,DC,0.00
8,DE,0.00
9,FL,414.00


---

4. For the list of states above that had $0 sales, generate a list of all the customers in those states, along with how much they have bought from us since then.

Return:
- cust_id
- first
- last
- addr
- city
- st
- zip
- the customer's total sales from all months after the month from question 2

Order the results by cust_id, ascending

In [11]:
# Note: Although this implementation answered the question, it is very complex are hard to interpret. In the future, I would to 
# choose to improve it by reducing the number of CTEs and considering the construction of another view.
sql = """
WITH 
LowestSalesMonth as
    (
    SELECT year_month, MIN(Sales) as Sales
    FROM vSalesByYearMonth
    ),
NoSaleStates as
    (
    WITH
    StatesWithSales as
        (
        SELECT st, SUM(qty*unit_price) as Sales
        FROM vOrderDatesParsed
        JOIN tOrderDetail USING(order_id)
        JOIN tProd USING(prod_id)
        JOIN tCust USING(cust_id)
        JOIN tZip USING(zip)
        WHERE year_month = (SELECT year_month FROM LowestSalesMonth)
        GROUP BY st
        )
    SELECT st
    FROM tState
    LEFT JOIN StatesWithSales USING(st)
    WHERE Sales IS NULL
    ORDER BY st ASC
    )
SELECT cust_id, first, last, addr, city, st, zip, SUM(qty*unit_price) as Sales
FROM tCust
JOIN vOrderDatesParsed USING(cust_id)
JOIN tOrderDetail USING(order_id)
JOIN tProd USING(prod_id)
JOIN tZip USING(zip)
WHERE st IN NoSaleStates
    AND year_month > (SELECT year_month FROM LowestSalesMonth)
GROUP BY cust_id
ORDER BY cust_id ASC
;"""
s.RunQuery(sql)

,cust_id,first,last,addr,city,st,zip,Sales
0,3,Dodonna,Garza,3639 Briarwood Court,Tarzan,TX,79783,364.90
1,6,Motti,Hunt,1919 Smith Street,Hamden,CT,06514,5465.67
2,7,Mace Windu,Woodward,5655 Jefferson Court,Milan,NM,87021,9422.33
3,11,Senator Ask Aak,Chen,2027 Pheasant Run,Wayne,NJ,07470,11028.03
4,12,Jabba,Oneill,9371 Walnut Street,Swampscott,MA,01907,9848.63
...,...,...,...,...,...,...,...,...
113,297,Tey How,Rodriguez,9164 Madison Street,Brick,NJ,08723,4347.75
114,301,Rabe,Mitchell,8021 Heather Lane,Pindall,AR,72669,8828.98
115,302,Taun We,Miller,4011 Sycamore Drive,Yellow Jacket,CO,81335,11850.42
116,305,Dodonna,Francis,8070 Briarwood Court,Buckland,AK,99727,4878.76


---

5) Get a list of customers who did not purchase anything in the most recent month of data, along with their average sales for all months prior.

Return:

- customer id
- name, address, zip, city, st (abbreviation is fine)
- total sales for most recent month (to confirm they are all 0)
- average sales for all months prior

Order the results with the largest average monthly sales on top.

In [12]:
# Note: Although this implementation answered the question, it is very complex are hard to interpret. In the future, I would to 
# choose to improve it by reducing the number of CTEs and adding a parameter for the date in RecentMonthCusts for easier reuse.
sql = """
WITH
NumMonths as
    (
    SELECT count(DISTINCT year_month) as num_months
    FROM vOrderDatesParsed
    ),
RecentMonthCusts as
    (
    SELECT cust_id, first, last, addr, zip, SUM(qty*unit_price) as RecentMonthSales
    FROM tCust
    JOIN tOrder USING(cust_id)
    JOIN tOrderDetail USING (order_id)
    JOIN tProd USING (prod_id)
    WHERE date LIKE '%2021-10-__%'
    GROUP BY cust_id
    ),
AllCustsPrevMonths as
    (
    WITH 
    PreviousSales as
        (
        SELECT cust_id, first, last, addr, zip, date, SUM(qty*unit_price) as CustPrevSalesSum
        FROM tCust
        JOIN tOrder USING(cust_id)
        JOIN tOrderDetail USING (order_id)
        JOIN tProd USING (prod_id)
        WHERE date NOT LIKE '%2021-10-__%'
        GROUP BY cust_id
        )
    SELECT cust_id, first, last, addr, zip, CustPrevSalesSum/(num_months-1) as CustPrevSalesAvg
    FROM PreviousSales
    JOIN NumMonths
    )
SELECT cust_id,first,last, addr, zip, city, st, IFNULL(RecentMonthSales, 0) as RecentSales, CustPrevSalesAvg
FROM AllCustsPrevMonths
LEFT JOIN RecentMonthCusts USING(cust_id,first,last,addr,zip)
JOIN tZip USING(zip)
WHERE RecentSales == 0
ORDER BY CustPrevSalesAvg DESC
;"""
s.RunQuery(sql)

,cust_id,first,last,addr,zip,city,st,RecentSales,CustPrevSalesAvg
0,39,Unkar Plutt,Woodward,5772 4th Street,51650,Riverton,IA,0,590.661212
1,46,Captain Antilles,Walker,8516 Pheasant Run,20005,Washington,DC,0,561.206667
2,152,Unkar Plutt,Schmidt,9546 Brookside Drive,13623,Chippewa Bay,NY,0,533.290000
3,207,Jobal,Mitchell,1198 West Avenue,38629,Falkner,MS,0,519.972424
4,202,Gold Leader,Elliott,9326 Sycamore Street,25938,Victor,WV,0,509.008182
...,...,...,...,...,...,...,...,...,...
178,142,Mace Windu,Greene,9572 9th Street,61957,Windsor,IL,0,43.382121
179,243,Darth Maul,Walters,7851 Magnolia Court,58046,Hope,ND,0,39.101212
180,241,Bala-Tik,Zhang,1559 Lake Avenue,93673,Traver,CA,0,36.436364
181,233,Clone Commander Cody,Benson,4836 Front Street,28390,Spring Lake,NC,0,25.335152


---
6) Are there any products we haven't sold at least 1 of each month?

If so, return:
 
- prod_id
- prod_desc
- unit_price
- year_month

Order the results by prod_id, year_month, both ascending

In [13]:
sql = """
WITH 
AllProds as
    (
    SELECT *, year_month
    FROM tProd
    CROSS JOIN (SELECT DISTINCT(year_month) FROM vOrderDatesParsed)
    ),
ProdsSold as
    (
    SELECT DISTINCT(prod_id), year_month
    FROM vOrderDatesParsed
    JOIN tOrderDetail USING(order_id)
    ORDER BY prod_id,year_month
    )
SELECT prod_id,prod_desc,unit_price,year_month
FROM AllProds
LEFT JOIN ProdsSold USING(prod_id,year_month)
WHERE ProdsSold.prod_id IS NULL
ORDER BY prod_id ASC, year_month ASC
;"""
s.RunQuery(sql)

,prod_id,prod_desc,unit_price,year_month
0,300,Washer,0.10,2019-09
1,301,Nail,0.25,2019-09
2,305,Bradawl,1.99,2019-09
3,307,Sandpaper,3.00,2019-03
4,308,Screwdriver,3.00,2019-05
5,309,Chisel,4.99,2019-07
6,310,Scraper,7.99,2019-04
7,312,Plane,10.99,2019-07
8,313,Wrench,11.00,2019-06
9,314,Mallet,12.00,2019-06


---
7) What are our top 5 selling products (in terms of total dollars sold)?

Return:

- prod_id
- prod_desc
- unit_price
- total sales (i.e. sum of qty * unit price)

Order the results by total sales, descending

In [14]:
sql = """
SELECT prod_id, prod_desc, unit_price, SUM(qty*unit_price) as TotalSales
FROM tOrderDetail
JOIN tProd USING(prod_id)
GROUP BY prod_id
ORDER BY TotalSales DESC
LIMIT 5
;"""
s.RunQuery(sql)

,prod_id,prod_desc,unit_price,TotalSales
0,329,Chainsaw,499.99,943481.13
1,328,Workbench,300.00,633300.00
2,327,Ladder,80.00,154000.00
3,326,Drill,69.00,131652.00
4,325,Toolbox,50.00,100850.00


---

8) What month did we have our highest sales, and what was our top selling product that month? (All in terms of dollars).

Return:

- year_month (as a single column, with values like YYYY-MM)
- prod_id
- prod_desc
- The total sales for that month, for all products
- The total sales for that month, for the top selling product only
- The total quantity for that month, for the top selling product only

In [15]:
sql = """
WITH
MaxSalesMonth as 
    (
    SELECT year_month, MAX(Sales) as Sales
    FROM vSalesByYearMonth
    )
SELECT year_month, prod_id, prod_desc, Sales, SUM(qty*unit_price) as ProdSales, SUM(qty) as QtySold
FROM tOrderDetail
JOIN vOrderDatesParsed USING(order_id)
JOIN tProd USING(prod_id)
JOIN MaxSalesMonth USING(year_month)
GROUP BY prod_id
ORDER BY ProdSales DESC
LIMIT 1
;"""
s.RunQuery(sql)

,year_month,prod_id,prod_desc,Sales,ProdSales,QtySold
0,2021-10,329,Chainsaw,251768.12,86998.26,174
